# Reddit Sentiment Analysis

In [ ]:
import pandas as pd

data = pd.read_csv("../input/cricket-on-reddit/reddit_cricket.csv")

print(data.shape)
data.head

# Preprocess The Comments
When dealing with raw or uncleaned data, we would first have to preprocess or “clean” it before modelling or in our case, to apply a sentiment analysis.

In our instance, we would include the following steps for our preprocessing:
* Removing Emojis
* Tokenizing, removing links etc.
* Removing stopwords
* Normalizing words via lemmatizing

In [ ]:
import emoji
import re

data['emojiless'] = data['body'].apply(lambda x: emoji.get_emoji_regexp().sub(u'', str(x)))
# data['emojiless']

In [ ]:
from nltk.tokenize import RegexpTokenizer  
tokenizer = RegexpTokenizer('\w+|\$[\d\.]+http\S+')
data['token'] = data['emojiless'].apply(lambda x: tokenizer.tokenize(x))
# print(data['token'])
data['lower_token'] = data['token'].apply(lambda x: [word.lower() for word in x])
# data['lower_token']

In [ ]:
from nltk.corpus import stopwords
import en_core_web_sm
nlp = en_core_web_sm.load()
all_stopwords = nlp.Defaults.stop_words

data['token_without_sw'] = data['lower_token'].apply(lambda x: [word for word in x if not word in all_stopwords and len(word)>2 and not word in ["www", "http", "https", "reddit", "cricket", "nan"]])
# data['token_without_sw']

The last pre-processing step is one of **Lemmatizing** or **Stemming**.

Both processes are used to trim words down to their root words. However, stemming might return a root word that is not an actual word whereas, lemmatizing returns a root word that is an actual language word.

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

data['lemmatizer_tokens'] = data['token_without_sw'].apply(lambda x: [lemmatizer.lemmatize(w) for w in x])
# print(data['lemmatizer_tokens'])

For visualization purposes, we can see that the number of words pre and post-processing has reduced dramatically!

In [ ]:
print(f'Original number of words: ', sum([len(str(x)) for x in data['body']]))
print(f'Emojiless number of words: ', sum([len(str(x)) for x in data['emojiless']]))
print(f'Tokenized number of words: ', sum([len(str(x)) for x in data['lower_token']]))
print(f'Stopwordless number of words: ', sum([len(str(x)) for x in data['token_without_sw']]))
print(f'Lemmatized number of words: ', sum([len(str(x)) for x in data['lemmatizer_tokens']]))


# Apply A Sentiment Analyzer (VADER)

Once we obtain our cleaned output, we would calculate each tokenized word's polarity scores using the **VADER** (Valence Aware Dictionary for Sentiment Reasoning) model.

The polarity scores measure the positivity and negativity for each word. We are mostly interested in the **compound score**, which is normalized to be between -1 (most extreme negative sentiment) and +1 (most extreme positive sentiment). This provides a single unidimensional measure of sentiment for a given word.

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import numpy as np

sia = SIA()
results = []

data['pool_score'] = data['lemmatizer_tokens'].apply(lambda x: [sia.polarity_scores(w).get('compound') for w in x]) 
# data['pool_score']
data['label'] = 0
data.loc[data['pool_score'].apply(lambda x: np.mean(x) > 0.1), 'label'] = 1
data.loc[data['pool_score'].apply(lambda x: np.mean(x) < 0.1), 'label'] = -1
# data['label']

# Representation Of Sentiment Results

Using a quick value count, we can see that the sentiment for most of the words is nagative; they are also more positive than neutral in sentiment.

In [ ]:
print(data.label.value_counts())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8,8))

counts = data.label.value_counts(normalize=True) * 100

sns.barplot(x=counts.index, y=counts, ax=ax)

ax.set_xticklabels(['Negative', 'Neutral', 'Positive'])
ax.set_ylabel('Percentage')

plt.show()

# Simple Visualizations

Lastly, before ending this post, we can perform some simple visualizations and explore the word frequency of positive and negative words.

Frequency distribution of the 20 most common positive words:

In [ ]:
from nltk import FreqDist 

all_words = " ".join(x for x in data['lemmatizer_tokens'].apply(lambda x: " ".join(x))).split(" ")
positive_words = " ".join(x for x in data[data['label'] == 1]['lemmatizer_tokens'].apply(lambda x: " ".join(x))).split(" ")
negative_words = " ".join(x for x in data[data['label'] == -1]['lemmatizer_tokens'].apply(lambda x: " ".join(x))).split(" ")

pos_freq = FreqDist(positive_words).most_common(5)
neg_freq = FreqDist(negative_words).most_common(5)
print('pos_freq\n', pos_freq)
print('neg_freq\n', neg_freq)

We can also visualize them via WordClouds.

In [ ]:
from wordcloud import WordCloud

wordcloud_total = WordCloud(background_color="white").generate(" , ".join(all_words))
wordcloud_posit = WordCloud(background_color="white").generate(" , ".join(positive_words))
wordcloud_negat = WordCloud(background_color="white").generate(" , ".join(negative_words))

plt.imshow(wordcloud_total, interpolation='bilinear')
plt.axis('off')
plt.show()

plt.imshow(wordcloud_posit, interpolation='bilinear')
plt.axis('off')
plt.show()

plt.imshow(wordcloud_negat, interpolation='bilinear')
plt.axis('off')
plt.show()